In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import numpy as np

In [3]:
dataset = Planetoid( root = '~/tmp/Cora', name = 'Cora' )
data = dataset[0]
dataset.num_features, dataset.num_classes

(1433, 7)

In [4]:
data.edge_index.size()

torch.Size([2, 10556])

In [5]:
data.train_mask, data.test_mask, np.sum( np.array( data.test_mask ) )             # 划分好的训练集测试集

(tensor([ True,  True,  True,  ..., False, False, False]),
 tensor([False, False, False,  ...,  True,  True,  True]),
 1000)

In [6]:
data.x.size(), data.y.size()                       # 节点的特征矩阵和标签的

(torch.Size([2708, 1433]), torch.Size([2708]))

In [7]:
class GNN( nn.Module ):
    def __init__( self ):
        super( GNN, self ).__init__()
        self.conv1 = GCNConv( dataset.num_features, 16 )                # ( 输入特征维度, 隐藏神经元个数 )
        self.conv2 = GCNConv( 16, dataset.num_classes )
        
    def forward( self, x, edge_index ):
        x = self.conv1( x, edge_index )
        x = F.relu( x )
        x = self.conv2( x, edge_index )
        return F.log_softmax( x, dim = 1 )


In [8]:
net = GNN()
optimizer = torch.optim.Adam( net.parameters(), lr = 0.01, weight_decay = 5e-4 ) # weight_decay 正则化，防止模型过拟合

In [9]:
for epoch in range( 200 ):
    net.train()
    optimizer.zero_grad()
    out = net( data.x, data.edge_index )                                        # 节点特征和边特征
    loss = F.nll_loss( out[data.train_mask], data.y[data.train_mask] )              # 仅计算训练集的损失
    loss.backward()
    optimizer.step()

net.eval()
pred = net( data.x, data.edge_index )[ data.test_mask ]

pred.size()

torch.Size([1000, 7])

In [12]:
max_index = torch.argmax( pred, dim = 1 )
test_true = data.y[ data.test_mask ]
r = 0
for i in range( len( max_index ) ):
    if max_index[i] == test_true[i]:
        r += 1
print( 'accuracy: {}%'.format( r * 100 / len( test_true ) ) )

accuracy: 80.4%
